In [1]:
import cv2
from ultralytics import YOLOWorld
import supervision as sv

In [3]:
image = cv2.imread("cars_images/images/437.jpg")
model = YOLOWorld("yolov8x-worldv2.pt")

# Define custom classes
model.set_classes(["license_plate", ""])
# Создаем датасет
dataset = sv.DetectionDataset(classes=model.names, images=['437.jpg'], annotations={'437.jpg': None})

# Execute prediction for specified categories on an image
results = model.predict(image, conf=0.1)

# Получение детекций для Supervision
detections = sv.Detections.from_ultralytics(results[0])
# фильтрация детекций по ширине bounding-box-а
w = detections.xyxy[:, 2] - detections.xyxy[:, 0]
h = detections.xyxy[:, 3] - detections.xyxy[:, 1]
detections = detections[(w < 500)]
#  Non-Maximum Suppression (NMS) постобработка для удаления дублирующихся прямоугольников
filtered_detections = detections.with_nms(threshold=0.5)

# Визуализация боксов и подписей
annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

# вывод доп информации (уверенность)
labels = [
    f"{class_name} {confidence:.2f}"
    for class_name, confidence
    in zip(filtered_detections['class_name'], filtered_detections.confidence)
]
annotated_image = annotator.annotate(scene=image, detections=filtered_detections)
annotated_image = label_annotator.annotate(scene=annotated_image, detections=filtered_detections, labels=labels)

cv2.imwrite("annotated_437.jpg", annotated_image)

# Вносим в датасет инфу по детекциям для конкретного файла
dataset.annotations['437.jpg'] = filtered_detections

# Экспортируем в YOLO-формат
dataset.as_yolo(
    images_directory_path="license_plate_dataset/images",
    annotations_directory_path="license_plate_dataset/labels",
    data_yaml_path="license_plate_dataset/data.yaml"
)



0: 512x640 2 license_plates, 126.0ms
Speed: 2.8ms preprocess, 126.0ms inference, 240.3ms postprocess per image at shape (1, 3, 512, 640)
